# Results p. 2
## Fixation Temporal Alignment
### Comparing RTO & RTD between Detectors

In [1]:
import os

import cv2
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from docutils.nodes import legend
from numpy.ma.extras import vstack
from plotly.subplots import make_subplots
import plotly.io as pio

import peyes

import analysis.utils as u
from analysis._article_results.hfc._helpers import *
import analysis.statistics.channel_time_diffs as ctd
import analysis.statistics.channel_sdt as ch_sdt

pio.renderers.default = "browser"

C:\Users\jonathanni\PycharmProjects\pEYES\venv\Lib\site-packages\statsmodels\tools\tools.py:6: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.0)
  import scipy.linalg


## Load Data
The `time_diffs` table describes the difference in onset/offset timings between the ground truth and the predicted fixations.  
The `thresholded_time_diffs` table is a subset of `time_diffs` where only the time-diffs whose absolute value is less than or equal to the `MATCHING_THRESHOLD` are kept.

In [2]:
MATCHING_THRESHOLD = 20     # samples; matches with time-diff greater than this value are not considered "hits"

time_diffs = ctd.load(
    DATASET_NAME,
    PROCESSED_DATA_DIR,
    label=1,        # EventLabelEnum.FIXATION.value
    stimulus_type=STIMULUS_TYPE,
)

thresholded_time_diffs = time_diffs.map(lambda vals: np.fromiter(filter(lambda v: abs(v) <= MATCHING_THRESHOLD, vals), dtype=int))

# drop human annotators that aren't GT1 or GT2 (and aren't detectors)
annotators_to_drop = [ann for ann in thresholded_time_diffs.columns.get_level_values(u.PRED_STR).unique() if ann not in [GT1, GT2] and ann not in DETECTORS.keys()]
thresholded_time_diffs.drop(columns=annotators_to_drop, level=u.GT_STR, inplace=True)
thresholded_time_diffs.drop(columns=annotators_to_drop, level=u.PRED_STR, inplace=True)

thresholded_time_diffs.T

channel_type                                                      onset  \
trial_id gt pred                                                          
1        MN remodnav  [0, -2, 5, 2, -2, -7, -2, -4, -2, -2, -6, -2, ...   
            idvt      [0, -13, 11, -7, -8, -2, -7, -8, -5, -5, -5, -...   
            RA        [0, 0, -4, 3, 0, 1, 3, 0, -3, 0, -2, -4, -1, 2...   
            engbert   [3, -1, -3, 1, -4, -3, -2, 1, -3, -4, 1, 0, -3...   
            ivt       [1, 13, 1, 0, 20, 18, 6, 4, 3, -17, 5, 2, 18, ...   
...                                                                 ...   
10       RA ivt       [1, 15, -2, -16, 19, 4, 6, -1, 4, 10, 7, 3, -1...   
            nh        [0, -9, -4, -5, -7, -6, -3, -2, 2, -4, 8, -5, ...   
            idt       [0, -13, -8, -6, -6, -8, -4, -6, -8, -6, -6, -...   
            ivvt      [2, -16, 19, 4, -1, 5, 8, -16, -16, 5, 13, 10,...   
            MN        [0, -5, 0, 1, 1, 2, 1, 1, 5, 1, -1, -3, -1, 2,...   

channel_type                                                     offset  
trial_id gt pred                                                         
1        MN remodnav  [3, 3, 0, 1, 4, 3, 2, 6, 8, 2, 6, 4, 3, 3, 6, ...  
            idvt      [2, 6, 4, 5, 5, 2, 4, 4, 2, 6, 11, 3, 7, 6, 5,...  
            RA        [1, 1, 0, 0, -1, 2, 2, 1, 1, -1, -1, 1, 2, 7, ...  
            engbert   [2, 2, 2, 1, 2, 2, 0, 2, 2, 3, 1, 1, 2, 1, 7, ...  
            ivt       [-8, -6, -9, -13, -12, 19, -3, -6, -10, -1, -1...  
...                                                                 ...  
10       RA ivt       [-9, 2, -6, 3, -2, -11, -14, -4, 1, 0, 0, -7, ...  
            nh        [-1, -2, -2, -1, -1, 0, -1, 0, -2, -1, -2, 0, ...  
            idt       [5, 4, 5, 6, 5, 5, 3, 4, 4, 3, 3, 4, 1, 4, 4, ...  
            ivvt      [-15, -2, -8, 3, -2, -5, 0, -7, 3, -1, -15, 0,...  
            MN        [0, -1, -1, -1, -4, 0, 1, -1, -2, 0, 0, 0, -1,...  

[160 rows x 2 columns]

## Compute Metrics

_Comments on the calculations:_  
(1) `num_gt_events` is the number of onsets/offsets across all trials, for the human annotator of every _(human, algorithm)_ pair.  
(2) `num_pred_events` is the number of onsets/offsets across all trials, for the algorithm of every _(human, algorithm)_ pair (or the _other human_ annotator).  
  
(3) `num_matches` is the number of matching onsets/offsets between the ground truth and the predicted events.  
  
(4) `hit_rate` (_recall_) is the percentage of matched GT onsets/offsets out of all GTs.  
(5) `ppv` (_precision_) is the percentage of matched PRED onsets/offsets out of all PREDs.  
(6) `f1` is the harmonic mean of `hit_rate` and `ppv`.
  
(7) `RTO` is the mean difference in onset/offset timings (measured in samples).  
(8) `RTD` is the standard deviation of difference in onset/offset timings (measured in samples).  

In [3]:
def calculate_alignment_metrics(td: pd.DataFrame) -> pd.DataFrame:
    num_gt_events, num_pred_events = _calc_num_events()
    aggregate_td = td.unstack(0).unstack(0).apply(np.hstack, axis=1).apply(lambda vals: vals[~np.isnan(vals)]).rename("time_diffs")
    aggregate_td = aggregate_td.reindex(u.sort_labelers(aggregate_td.index.get_level_values(u.PRED_STR)), level=u.PRED_STR)
    num_matches = aggregate_td.apply(len).rename('num_matches')     # number of matching onsets/offsets between the ground truth and the predicted events.
    
    hit_rate = (num_matches / num_gt_events).rename('hit_rate')
    ppv = (num_matches / num_pred_events).rename('ppv')
    f1 = (2 * (hit_rate * ppv) / (hit_rate + ppv)).rename('f1')
    rto = aggregate_td.apply(np.mean).rename('RTO')
    rtd = aggregate_td.apply(np.std).rename('RTD')
    
    alignment = pd.concat([num_gt_events, num_pred_events, num_matches, hit_rate, ppv, f1, rto, rtd], axis=1).unstack(
        [u.GT_STR, u.PRED_STR]).stack(0, future_stack=True)
    alignment.reindex(columns=u.sort_labelers(alignment.columns.get_level_values(u.PRED_STR).unique()), level=1)
    alignment.index.names = [peyes.constants.CHANNEL_TYPE_STR, peyes.constants.METRIC_STR]
    alignment.columns.names = [u.GT_STR, u.PRED_STR]
    return alignment


def _calc_num_events() -> (pd.Series, pd.Series):
    # Loads the channel-SDT data and extracts the number of GT and PRED events (onsets/offsets) for each labeler-detector pair.
    num_events = ch_sdt.load(
        dataset_name=DATASET_NAME,
        output_dir=PROCESSED_DATA_DIR,
        label=1,
        stimulus_type=STIMULUS_TYPE,
        channel_type=None,
    )
    num_events = num_events.loc[(slice(None), ['P', 'PP'], 0)]      # keep only the positive counts
    num_events = num_events.droplevel(peyes.constants.THRESHOLD_STR)
    num_events = num_events.stack([u.GT_STR, u.PRED_STR], future_stack=True).sum(axis=1)
    num_events = num_events.reorder_levels([u.GT_STR, u.PRED_STR, peyes.constants.METRIC_STR, peyes.constants.CHANNEL_TYPE_STR]).sort_index()
    
    num_gt_events = num_events.xs('P', level=peyes.constants.METRIC_STR).rename('num_gt')   # number of onsets/offsets across all trials, for the human annotator of every _(human, algorithm)_ pair.
    num_pred_events = num_events.xs('PP', level=peyes.constants.METRIC_STR).rename('num_pred')  # number of onsets/offsets across all trials, for the algorithm of every _(human, algorithm)_ pair (or the _other human_ annotator).
    return num_gt_events, num_pred_events

In [4]:
alignment_metrics = calculate_alignment_metrics(thresholded_time_diffs)
alignment_metrics

gt                           DN                                            \
pred                         IH     JB     JF     JV     KH     MN     MS   
channel_type metric                                                         
offset       num_gt       412.0  412.0  412.0  412.0  412.0  412.0  412.0   
             num_pred     435.0  428.0  465.0  423.0  426.0  411.0  385.0   
             num_matches    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             hit_rate       NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             ppv            NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             f1             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             RTO            NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             RTD            NaN    NaN    NaN    NaN    NaN    NaN    NaN   
onset        num_gt       412.0  412.0  412.0  412.0  412.0  412.0  412.0   
             num_pred     435.0  428.0  465.0  423.0  426.0  411.0  385.0   
             num_matches    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             hit_rate       NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             ppv            NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             f1             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             RTO            NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             RTD            NaN    NaN    NaN    NaN    NaN    NaN    NaN   

gt                                             ...     TC                \
pred                         PZ     RA     RH  ...     PZ     RA     RH   
channel_type metric                            ...                        
offset       num_gt       412.0  412.0  412.0  ...  439.0  439.0  439.0   
             num_pred     470.0  416.0  421.0  ...  470.0  416.0  421.0   
             num_matches    NaN    NaN    NaN  ...    NaN    NaN    NaN   
             hit_rate       NaN    NaN    NaN  ...    NaN    NaN    NaN   
             ppv            NaN    NaN    NaN  ...    NaN    NaN    NaN   
             f1             NaN    NaN    NaN  ...    NaN    NaN    NaN   
             RTO            NaN    NaN    NaN  ...    NaN    NaN    NaN   
             RTD            NaN    NaN    NaN  ...    NaN    NaN    NaN   
onset        num_gt       412.0  412.0  412.0  ...  439.0  439.0  439.0   
             num_pred     470.0  416.0  421.0  ...  470.0  416.0  421.0   
             num_matches    NaN    NaN    NaN  ...    NaN    NaN    NaN   
             hit_rate       NaN    NaN    NaN  ...    NaN    NaN    NaN   
             ppv            NaN    NaN    NaN  ...    NaN    NaN    NaN   
             f1             NaN    NaN    NaN  ...    NaN    NaN    NaN   
             RTO            NaN    NaN    NaN  ...    NaN    NaN    NaN   
             RTD            NaN    NaN    NaN  ...    NaN    NaN    NaN   

gt                                                                             
pred                     engbert    idt   idvt     ivt   ivvt     nh remodnav  
channel_type metric                                                            
offset       num_gt        439.0  439.0  439.0   439.0  439.0  439.0    439.0  
             num_pred      580.0  257.0  261.0  1033.0  685.0  350.0    425.0  
             num_matches     NaN    NaN    NaN     NaN    NaN    NaN      NaN  
             hit_rate        NaN    NaN    NaN     NaN    NaN    NaN      NaN  
             ppv             NaN    NaN    NaN     NaN    NaN    NaN      NaN  
             f1              NaN    NaN    NaN     NaN    NaN    NaN      NaN  
             RTO             NaN    NaN    NaN     NaN    NaN    NaN      NaN  
             RTD             NaN    NaN    NaN     NaN    NaN    NaN      NaN  
onset        num_gt        439.0  439.0  439.0   439.0  439.0  439.0    439.0  
             num_pred      580.0  257.0  261.0  1033.0  685.0  350.0    425.0  
             num_matches     NaN    NaN    NaN     NaN    NaN    NaN  

#### Annotator RA

In [5]:
ra_onset_alignments = alignment_metrics.xs(
    peyes.constants.ONSET_STR, level=peyes.constants.CHANNEL_TYPE_STR, axis=0
).stack(u.GT_STR, future_stack=True).xs('RA', level=u.GT_STR).dropna(axis=1, how='all')
ra_onset_alignments = ra_onset_alignments.reindex(
    columns=u.sort_labelers(ra_onset_alignments.columns.get_level_values(u.PRED_STR).unique()), level=1
)

ra_onset_alignments.loc[:, [GT2] + list(DETECTORS.keys())]

pred,MN,ivt,ivvt,idt,idvt,engbert,nh,remodnav
metric,,,,,,,,
num_gt,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000
num_pred,411.000000,1033.000000,685.000000,257.000000,261.000000,580.000000,350.000000,425.000000
num_matches,390.000000,260.000000,159.000000,218.000000,229.000000,412.000000,323.000000,337.000000
hit_rate,0.937500,0.625000,0.382212,0.524038,0.550481,0.990385,0.776442,0.810096
ppv,0.948905,0.251694,0.232117,0.848249,0.877395,0.710345,0.922857,0.792941
f1,0.943168,0.358868,0.288828,0.647845,0.676514,0.827309,0.843342,0.801427
RTO,0.330769,3.465385,2.830189,-5.715596,-5.746725,-1.475728,-2.157895,-1.661721
RTD,3.543984,8.703379,10.337821,4.895405,4.774985,3.496447,3.997270,4.273108


In [6]:
ra_offset_alignments = alignment_metrics.xs(
    peyes.constants.OFFSET_STR, level=peyes.constants.CHANNEL_TYPE_STR, axis=0
).stack(u.GT_STR, future_stack=True).xs('RA', level=u.GT_STR).dropna(axis=1, how='all')
ra_offset_alignments = ra_offset_alignments.reindex(
    columns=u.sort_labelers(ra_offset_alignments.columns.get_level_values(u.PRED_STR).unique()), level=1
)

ra_offset_alignmentsra_onset_alignments = alignment_metrics.xs(
    peyes.constants.ONSET_STR, level=peyes.constants.CHANNEL_TYPE_STR, axis=0
).stack(u.GT_STR, future_stack=True).xs('RA', level=u.GT_STR).dropna(axis=1, how='all')
ra_offset_alignments = ra_offset_alignments.reindex(
    columns=u.sort_labelers(ra_offset_alignments.columns.get_level_values(u.PRED_STR).unique()), level=1
)

ra_offset_alignments.loc[:, [GT2] + list(DETECTORS.keys())]

pred,MN,ivt,ivvt,idt,idvt,engbert,nh,remodnav
metric,,,,,,,,
num_gt,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000
num_pred,411.000000,1033.000000,685.000000,257.000000,261.000000,580.000000,350.000000,425.000000
num_matches,390.000000,271.000000,159.000000,216.000000,230.000000,411.000000,320.000000,305.000000
hit_rate,0.937500,0.651442,0.382212,0.519231,0.552885,0.987981,0.769231,0.733173
ppv,0.948905,0.262343,0.232117,0.840467,0.881226,0.708621,0.914286,0.717647
f1,0.943168,0.374051,0.288828,0.641902,0.679468,0.825301,0.835509,0.725327
RTO,-0.433333,-3.051661,-4.352201,1.652778,2.200000,0.854015,-1.521875,1.524590
RTD,2.347812,7.486587,8.803329,5.756220,5.195650,2.476773,2.707540,3.913016


#### Annotator MN

In [7]:
mn_onset_alignments = alignment_metrics.xs(
    peyes.constants.ONSET_STR, level=peyes.constants.CHANNEL_TYPE_STR, axis=0
).stack(u.GT_STR, future_stack=True).xs('MN', level=u.GT_STR).dropna(axis=1, how='all')
mn_onset_alignments = mn_onset_alignments.reindex(
    columns=u.sort_labelers(mn_onset_alignments.columns.get_level_values(u.PRED_STR).unique()), level=1
)

mn_onset_alignments.loc[:, [GT1] + list(DETECTORS.keys())]

pred,RA,ivt,ivvt,idt,idvt,engbert,nh,remodnav
metric,,,,,,,,
num_gt,411.000000,411.000000,411.000000,411.000000,411.000000,411.000000,411.000000,411.000000
num_pred,416.000000,1033.000000,685.000000,257.000000,261.000000,580.000000,350.000000,425.000000
num_matches,390.000000,265.000000,161.000000,217.000000,227.000000,407.000000,324.000000,339.000000
hit_rate,0.948905,0.644769,0.391727,0.527981,0.552311,0.990268,0.788321,0.824818
ppv,0.937500,0.256534,0.235036,0.844358,0.869732,0.701724,0.925714,0.797647
f1,0.943168,0.367036,0.293796,0.649701,0.675595,0.821393,0.851511,0.811005
RTO,-0.330769,3.339623,3.447205,-6.069124,-6.101322,-2.022113,-2.432099,-1.955752
RTD,3.543984,9.029811,10.488030,4.515188,4.444801,2.503462,3.125713,3.428295


In [8]:
mn_offset_alignments = alignment_metrics.xs(
    peyes.constants.OFFSET_STR, level=peyes.constants.CHANNEL_TYPE_STR, axis=0
).stack(u.GT_STR, future_stack=True).xs('MN', level=u.GT_STR).dropna(axis=1, how='all')
mn_offset_alignments = mn_offset_alignments.reindex(
    columns=u.sort_labelers(mn_offset_alignments.columns.get_level_values(u.PRED_STR).unique()), level=1
)

mn_offset_alignments.loc[:, [GT1] + list(DETECTORS.keys())]

pred,RA,ivt,ivvt,idt,idvt,engbert,nh,remodnav
metric,,,,,,,,
num_gt,411.000000,411.000000,411.000000,411.000000,411.000000,411.000000,411.000000,411.000000
num_pred,416.000000,1033.000000,685.000000,257.000000,261.000000,580.000000,350.000000,425.000000
num_matches,390.000000,268.000000,161.000000,215.000000,231.000000,407.000000,324.000000,304.000000
hit_rate,0.948905,0.652068,0.391727,0.523114,0.562044,0.990268,0.788321,0.739659
ppv,0.937500,0.259439,0.235036,0.836576,0.885057,0.701724,0.925714,0.715294
f1,0.943168,0.371191,0.293796,0.643713,0.687500,0.821393,0.851511,0.727273
RTO,0.433333,-2.626866,-3.267081,2.581395,2.917749,1.299754,-1.111111,2.023026
RTD,2.347812,7.155179,9.369408,4.475424,4.372007,1.702311,2.506781,2.987275


## Statistical Analysis

In [9]:
statistics, pvalues, dunns, Ns = ctd.kruskal_wallis_dunns(thresholded_time_diffs, [GT1, GT2], multi_comp=MULTI_COMP)
statistics.index.names = pvalues.index.names = dunns.index.names = [peyes.constants.CHANNEL_TYPE_STR]

pd.concat([statistics, pvalues, pvalues <= ALPHA], axis=1, keys=['H', 'p', 'is_sig']).stack(1, future_stack=True)

H              p  is_sig
channel_type gt                                   
offset       MN  641.819298  2.213872e-135    True
             RA  587.513306  1.150824e-123    True
onset        MN  640.062992  5.298546e-135    True
             RA  576.197977  3.171519e-121    True

#### Post Hoc Analysis

In [10]:
post_hoc_onset = ctd.post_hoc_table(dunns, peyes.constants.ONSET_STR, [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)
post_hoc_onset

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.     ***     ***     ***     ***      ***
         RA      --    n.s.     ***     ***     ***     ***      ***
ivvt     MN  1.0000      --     ***     ***     ***     ***      ***
         RA  1.0000      --     ***     ***     ***     ***      ***
idt      MN  0.0000  0.0000      --    n.s.     ***     ***      ***
         RA  0.0000  0.0000      --    n.s.     ***     ***      ***
idvt     MN  0.0000  0.0000  1.0000      --     ***     ***      ***
         RA  0.0000  0.0000  1.0000      --     ***     ***      ***
engbert  MN  0.0000  0.0000  0.0000  0.0000      --    n.s.     n.s.
         RA  0.0000  0.0000  0.0000  0.0000      --    n.s.     n.s.
nh       MN  0.0000  0.0000  0.0000  0.0000  1.0000      --     n.s.
         RA  0.0000  0.0000  0.0000  0.0000  0.2982      --     n.s.
remodnav MN  0.0000  0.0000  0.0000  0.0000  1.0000  1.0000       --
         RA  0.0000  0.0000  0.0000  0.0000  1.0000  1.0000       --

In [11]:
post_hoc_offset = ctd.post_hoc_table(dunns, peyes.constants.OFFSET_STR, [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)
post_hoc_offset

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.     ***     ***     ***     ***      ***
         RA      --    n.s.     ***     ***     ***       *      ***
ivvt     MN  1.0000      --     ***     ***     ***    n.s.      ***
         RA  0.1884      --     ***     ***     ***    n.s.      ***
idt      MN  0.0000  0.0000      --    n.s.     ***     ***      ***
         RA  0.0000  0.0000      --    n.s.     ***     ***       **
idvt     MN  0.0000  0.0000  1.0000      --     ***     ***      ***
         RA  0.0000  0.0000  1.0000      --     ***     ***      ***
engbert  MN  0.0000  0.0000  0.0000  0.0000      --     ***      ***
         RA  0.0000  0.0000  0.0000  0.0000      --     ***      ***
nh       MN  0.0008  1.0000  0.0000  0.0000  0.0000      --      ***
         RA  0.0115  1.0000  0.0000  0.0000  0.0000      --      ***
remodnav MN  0.0000  0.0000  0.0008  0.0000  0.0001  0.0000       --
         RA  0.0000  0.0000  0.0035  0.0000  0.0004  0.0000       --

### RTO Distribution
#### (RTD is the s.t.d. around RTO) 

In [12]:
W, H = 600, 400
EVENT_TYPE = "Fixation"

temporal_alignment_fig = ctd.distributions_figure(
    thresholded_time_diffs, GT1, gt2=GT2, only_box=False, show_other_gt=True, share_x=True,
    colors={k: v[1] for k, v in LABELER_PLOTTING_CONFIG.items()},
    subplots_vspace=0.15,
)
temporal_alignment_fig.update_traces(width=0.9)     # make violins wider so there's less space between them

temporal_alignment_fig.update_layout(
    title=None,
    width=W, height=H,
    paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
    yaxis=dict(range=[-MATCHING_THRESHOLD, MATCHING_THRESHOLD], showgrid=False, zeroline=True, title='Δt (samples)'),
    yaxis2=dict(range=[-MATCHING_THRESHOLD, MATCHING_THRESHOLD], showgrid=False, zeroline=True, title='Δt (samples)'),
    margin=dict(l=10, r=10, b=10, t=20, pad=0),
    showlegend=False,
)

# update subplot titles:
for annotation in temporal_alignment_fig.layout.annotations:
    annotation.text = f"{EVENT_TYPE} {annotation.text}"
# temporal_alignment_fig.layout.annotations = []    # remove subtitles

# FIG_ID, PANEL_ID, IS_SUPP = 4, 'A', False
# save_fig(temporal_alignment_fig, FIG_ID, PANEL_ID, 'temporal-alignment_fixation', IS_SUPP)
temporal_alignment_fig.show()